In [109]:
import numpy as np
import pandas as pd
import datetime as dt
import os
from compass_smartsheet import CompassSmartsheet
import cisco_fiscal_calendar as fc

In [110]:
finbi_path = r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.pkl'
fins = pd.read_pickle(finbi_path)

In [111]:
# fins

In [112]:
compass_tracker_pickle_version = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'

In [113]:
tracker = pd.read_pickle(compass_tracker_pickle_version)

In [114]:
current_run_date = dt.datetime.today() #.isoformat()[:10]

In [115]:
file_run_date = current_run_date.isoformat()[:10]

In [116]:
thirty_days_ago = current_run_date + dt.timedelta(weeks = -4, days = -2)

In [117]:
thirty_days_ago = thirty_days_ago.isoformat()[:10]

In [118]:
thirty_days_ago

'2021-12-26'

In [119]:
eleven_months_ago = current_run_date + dt.timedelta(weeks = -48, days = -3)

In [120]:
eleven_months_ago = eleven_months_ago.isoformat()[:10]

In [121]:
eleven_months_ago

'2021-02-20'

In [122]:
# eleven_months_ago = today + dt.timedelta(weeks = -48, days = -3)

In [123]:
# eleven_months_ago = eleven_months_ago.isoformat()[:10]

In [124]:
caldf = fc.calendar_df

In [125]:
oldest_fiscal_week_id = caldf.loc[caldf['date'] == eleven_months_ago]['finbi_fiscal_week_id'].values[0].astype(str)

In [126]:
type(oldest_fiscal_week_id)

numpy.str_

In [127]:
current_run_date_str = current_run_date.isoformat()[:10]

In [128]:
current_fiscal_week_id = caldf.loc[caldf['date'] == current_run_date_str]['finbi_fiscal_week_id'].values[0]

In [129]:
current_fiscal_week_id

2022065

In [130]:
fins['FINBI Name'] = fins.loc[fins['CAV Name'].notnull()].apply(lambda x: x['GUID Name'] + '_' + x['CAV Name'], axis=1)

In [131]:
fins.loc[fins['Sales Level 2']=='GLOBAL ENTERPRISE SEGMENT','Sales Level 2']='GES'

In [132]:
fins['Sales Level 1'] = fins['Sales Level 1'].str.upper()
fins['Sales Level 2'] = fins['Sales Level 2'].str.upper()

In [133]:
str(current_fiscal_week_id)[:6]

'202206'

In [134]:
fiscal_month_id = str(current_fiscal_week_id)[:6]

In [135]:
fins['Report Month'] = fiscal_month_id

In [136]:
non_latam = ~(fins['Sales Level 2']=='LATIN AMERICA')

In [137]:
fin_sl1           = fins.loc[(fins['CAV Name'].notnull()) & (fins['Fiscal Week ID'] >= oldest_fiscal_week_id) ].groupby(['Sales Level 1'])[['FINBI Name']].nunique()

In [138]:
fin_ct            = fins.loc[(fins['CAV Name'].notnull()) & (fins['Fiscal Week ID'] >= oldest_fiscal_week_id) ].groupby(['Campaign Type'])[['FINBI Name']].nunique()
fin_sl1_ct        = fins.loc[(fins['CAV Name'].notnull()) & (fins['Fiscal Week ID'] >= oldest_fiscal_week_id) ].groupby(['Sales Level 1','Campaign Type'])[['FINBI Name']].nunique()
fin_sl1_sl2       = fins.loc[(fins['CAV Name'].notnull()) & (fins['Fiscal Week ID'] >= oldest_fiscal_week_id) ].groupby(['Sales Level 1','Sales Level 2'])[['FINBI Name']].nunique()
fin_sl1_sl2_ct    = fins.loc[(fins['CAV Name'].notnull()) & (fins['Fiscal Week ID'] >= oldest_fiscal_week_id) ].groupby(['Sales Level 1','Sales Level 2','Campaign Type'])[['FINBI Name']].nunique()
overall_numerator = fins.loc[(fins['CAV Name'].notnull()) & (fins['Fiscal Week ID'] >= oldest_fiscal_week_id) ][['FINBI Name']].nunique()[0]

In [139]:
fin_sl1_ct.reset_index(inplace=True)
fin_ct.reset_index(inplace=True)
fin_sl1.reset_index(inplace=True)
fin_sl1_sl2.reset_index(inplace=True)
fin_sl1_sl2_ct.reset_index(inplace=True)

In [140]:
fin_sl1_ct['Report Month'] = fiscal_month_id
fin_ct['Report Month'] = fiscal_month_id
fin_sl1['Report Month'] = fiscal_month_id
fin_sl1_sl2['Report Month'] = fiscal_month_id
fin_sl1_sl2_ct['Report Month'] = fiscal_month_id

In [141]:
fin_sl1_sl2

,Sales Level 1,Sales Level 2,FINBI Name,Report Month
0,AMERICAS,AMERICAS_MISC,3,202206
1,AMERICAS,AMERICAS_SP,67,202206
2,AMERICAS,CANADA,73,202206
3,AMERICAS,GES,299,202206
4,AMERICAS,LATIN AMERICA,105,202206
5,AMERICAS,US COMMERCIAL,318,202206
6,AMERICAS,US PS MARKET SEGMENT,118,202206
7,APJC,ANZ AREA,24,202206
8,APJC,APJ_SP,12,202206
9,APJC,ASEAN_AREA,21,202206


In [142]:
fin_sl1['Sales Level 1']        = fin_sl1['Sales Level 1'].str.upper()
fin_ct['Campaign Type']         = fin_ct['Campaign Type'].str.upper()
fin_sl1_ct['Sales Level 1']     = fin_sl1_ct['Sales Level 1'].str.upper()
fin_sl1_sl2['Sales Level 1']    = fin_sl1_sl2['Sales Level 1'].str.upper()
fin_sl1_sl2_ct['Sales Level 1'] = fin_sl1_sl2_ct['Sales Level 1'].str.upper()
fin_sl1_sl2_ct['Sales Level 2'] = fin_sl1_sl2_ct['Sales Level 2'].str.upper()

In [143]:
removed_campaigns = ['REACTIVE - ACC DESIGNATION','REACTIVE - PARTNER','REACTIVE - GROWTH INITIATIVE',
                     'PROACTIVE - TAC LEAD','PROACTIVE - CS RESEARCH','PROACTIVE - PRESALES ALIGN PITSTOP EFFORT']

In [144]:
tracker['Campaign']

0        COVERAGE PROGRAM - TELEMETRY
1        COVERAGE PROGRAM - TELEMETRY
2        COVERAGE PROGRAM - TELEMETRY
3        COVERAGE PROGRAM - TELEMETRY
4        COVERAGE PROGRAM - TELEMETRY
                     ...             
35552                        REACTIVE
35553                        REACTIVE
35554                        REACTIVE
35555                        REACTIVE
35556                        REACTIVE
Name: Campaign, Length: 35557, dtype: object

In [145]:
tracker.loc[(tracker['Sales Level 2'].notnull()) & (tracker['Sales Level 2'].str.contains('UKI', case=False)), 'Sales Level 2'] = 'EMEAR-UKI'

In [146]:
tracker.loc[(tracker['Sales Level 2'].notnull()) & (tracker['Sales Level 2'].str.contains('central', case=False)), 'Sales Level 2'] = 'EMEAR-CENTRAL'

In [147]:
tracker.loc[(tracker['Sales Level 2'].notnull()) & (tracker['Sales Level 2'].str.contains('north', case=False)), 'Sales Level 2'] = 'EMEAR-NORTH'

In [148]:
tracker.loc[(tracker['Sales Level 2'].notnull()) & (tracker['Sales Level 2'].str.contains('south', case=False)), 'Sales Level 2'] = 'EMEAR-SOUTH'

In [149]:
tracker.loc[(tracker['Sales Level 2'].notnull()) & (tracker['Sales Level 2'].str.contains('ANZ', case=False)), 'Sales Level 2'] = 'ANZ AREA'

In [150]:
tracker.loc[(tracker['Sales Level 2'].notnull()) & (tracker['Sales Level 2'].str.contains('japan', case=False)), 'Sales Level 2'] = 'JAPAN__'

In [151]:
tracker.loc[(tracker['Sales Level 2'].notnull()) & (tracker['Sales Level 2'].str.contains('global enter', case=False)), 'Sales Level 2'] = 'GES'

In [152]:
trkr_condition1A = (tracker['Readout_Date']>= eleven_months_ago)
trkr_condition1B = (tracker['Readout_Date']<= thirty_days_ago)
trkr_condition2 = ~(tracker['Campaign'].isin(removed_campaigns))
trkr_condition3 = ~(tracker['Readout Delivery Status'].isin(['Declined','Postponed']))
#trkr_condition4 = ~(tracker['Sales Level 2']=='LATAM')

In [153]:
compass_sl1         = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Sales Level 1'])[['Customer Name']].nunique()
compass_ct          = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Campaign Type'])[['Customer Name']].nunique()
compass_sl1_ct      = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Sales Level 1','Campaign Type'])[['Customer Name']].nunique()
compass_sl1_sl2     = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Sales Level 1','Sales Level 2'])[['Customer Name']].nunique()
compass_sl1_sl2_ct  = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Sales Level 1','Sales Level 2','Campaign Type'])[['Customer Name']].nunique()
overall_denominator = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ][['Customer Name']].nunique()[0]

In [154]:
compass_sl1_sl2

Customer Name
Sales Level 1 Sales Level 2                      
AMERICAS      AMERICAS_SP                      92
              CANADA                          147
              GES                             568
              LATIN AMERICA                   142
              US COMMERCIAL                  1157
              US PS MARKET SEGMENT            308
APJC          ANZ AREA                         79
              APJ_SP                           26
              ASEAN_AREA                       55
              GREATER_CHINA                    81
              INDIA_AREA                       57
              JAPAN__                          63
              ROK_AREA                         15
EMEAR-REGION  EMEAR-CENTRAL                    93
              EMEAR-NORTH                      83
              EMEAR-SOUTH                      72
              EMEAR-UKI                        96
              EMEAR_GERMANY                    37
              EMEAR_MEA                        81
              EMEAR_SP                         21

In [155]:
# ### Sanity Check
# tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('UKI', case=False)), 'Lvl2 (Region)'] = 'EMEAR-UKI'

In [156]:
compass_sl1.reset_index(inplace=True)
compass_ct.reset_index(inplace=True)
compass_sl1_ct.reset_index(inplace=True)
compass_sl1_sl2.reset_index(inplace=True)
compass_sl1_sl2_ct.reset_index(inplace=True)

In [157]:
compass_sl1.rename(columns = {'Lvl1':'Sales Level 1','Customer Name':'Compass Name'}, inplace=True)
compass_ct.rename(columns = {'Customer Name':'Compass Name'}, inplace=True)
compass_sl1_ct.rename(columns = {'Lvl1':'Sales Level 1','Customer Name':'Compass Name'}, inplace=True)
compass_sl1_sl2.rename(columns = {'Lvl1':'Sales Level 1','Customer Name':'Compass Name'}, inplace=True)
compass_sl1_sl2_ct.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2','Customer Name':'Compass Name'}, inplace=True)

In [158]:
# fin_sl1_sl2_ct

In [159]:
combined_sl1 = pd.merge(fin_sl1, compass_sl1, on = ['Sales Level 1'])

In [160]:
combined_ct = pd.merge(fin_ct, compass_ct, on=['Campaign Type'])

In [161]:
fin_sl1_sl2

,Sales Level 1,Sales Level 2,FINBI Name,Report Month
0,AMERICAS,AMERICAS_MISC,3,202206
1,AMERICAS,AMERICAS_SP,67,202206
2,AMERICAS,CANADA,73,202206
3,AMERICAS,GES,299,202206
4,AMERICAS,LATIN AMERICA,105,202206
5,AMERICAS,US COMMERCIAL,318,202206
6,AMERICAS,US PS MARKET SEGMENT,118,202206
7,APJC,ANZ AREA,24,202206
8,APJC,APJ_SP,12,202206
9,APJC,ASEAN_AREA,21,202206


In [162]:
combined_sl1_ct = pd.merge(fin_sl1_ct, compass_sl1_ct, on = ['Sales Level 1','Campaign Type'])

In [163]:
combined_sl1_sl2 = pd.merge(fin_sl1_sl2, compass_sl1_sl2, on = ['Sales Level 1','Sales Level 2'])

In [164]:
combined_sl1_sl2_ct = pd.merge(fin_sl1_sl2_ct,compass_sl1_sl2_ct, on = ['Sales Level 1', 'Sales Level 2','Campaign Type'])

In [165]:
combined_sl1_sl2

,Sales Level 1,Sales Level 2,FINBI Name,Report Month,Compass Name
0,AMERICAS,AMERICAS_SP,67,202206,92
1,AMERICAS,CANADA,73,202206,147
2,AMERICAS,GES,299,202206,568
3,AMERICAS,LATIN AMERICA,105,202206,142
4,AMERICAS,US COMMERCIAL,318,202206,1157
5,AMERICAS,US PS MARKET SEGMENT,118,202206,308
6,APJC,ANZ AREA,24,202206,79
7,APJC,APJ_SP,12,202206,26
8,APJC,ASEAN_AREA,21,202206,55
9,APJC,GREATER_CHINA,15,202206,81


In [166]:
combined_sl1_sl2_ct['Conversion Ratio %'] = (combined_sl1_sl2_ct['FINBI Name'] / combined_sl1_sl2_ct['Compass Name']) * 100
combined_sl1_ct['Conversion Ratio %'] = (combined_sl1_ct['FINBI Name'] / combined_sl1_ct['Compass Name']) * 100
combined_sl1['Conversion Ratio %'] = (combined_sl1['FINBI Name'] / combined_sl1['Compass Name']) * 100
combined_sl1_sl2['Conversion Ratio %'] = (combined_sl1_sl2['FINBI Name'] / combined_sl1_sl2['Compass Name']) * 100
combined_ct['Conversion Ratio %'] = (combined_ct['FINBI Name'] / combined_ct['Compass Name']) * 100

In [167]:
combined_ct = combined_ct[['Report Month','Campaign Type', 'FINBI Name',  'Compass Name','Conversion Ratio %']]

In [168]:
combined_sl1_ct = combined_sl1_ct[['Report Month', 'Sales Level 1', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [169]:
combined_sl1 = combined_sl1[['Report Month', 'Sales Level 1', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [170]:
combined_sl1_sl2 = combined_sl1_sl2[['Report Month', 'Sales Level 1', 'Sales Level 2', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [171]:
combined_ct

,Report Month,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,202206,COVERAGE AT RENEWAL,4,277,1.444043
1,202206,COVERAGE PROGRAM,21,903,2.325581
2,202206,FIELD,770,1012,76.086957
3,202206,TARGETING,471,1087,43.330267


In [172]:
combined_sl1

,Report Month,Sales Level 1,FINBI Name,Compass Name,Conversion Ratio %
0,202206,AMERICAS,905,2400,37.708333
1,202206,APJC,116,374,31.016043
2,202206,EMEAR-REGION,149,482,30.912863


In [173]:
combined_sl1_sl2

,Report Month,Sales Level 1,Sales Level 2,FINBI Name,Compass Name,Conversion Ratio %
0,202206,AMERICAS,AMERICAS_SP,67,92,72.826087
1,202206,AMERICAS,CANADA,73,147,49.659864
2,202206,AMERICAS,GES,299,568,52.640845
3,202206,AMERICAS,LATIN AMERICA,105,142,73.943662
4,202206,AMERICAS,US COMMERCIAL,318,1157,27.484875
5,202206,AMERICAS,US PS MARKET SEGMENT,118,308,38.311688
6,202206,APJC,ANZ AREA,24,79,30.379747
7,202206,APJC,APJ_SP,12,26,46.153846
8,202206,APJC,ASEAN_AREA,21,55,38.181818
9,202206,APJC,GREATER_CHINA,15,81,18.518519


In [174]:
combined_sl1_sl2_ct = combined_sl1_sl2_ct[['Report Month', 'Sales Level 1', 'Sales Level 2', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

### Campaign Type Only

In [175]:
combined_ct.columns

Index(['Report Month', 'Campaign Type', 'FINBI Name', 'Compass Name',
       'Conversion Ratio %'],
      dtype='object')

### Sales Level 1 Only

In [176]:
combined_sl1

,Report Month,Sales Level 1,FINBI Name,Compass Name,Conversion Ratio %
0,202206,AMERICAS,905,2400,37.708333
1,202206,APJC,116,374,31.016043
2,202206,EMEAR-REGION,149,482,30.912863


### SL1 = Sales Level 1 and CT = Campaign Type

In [177]:
combined_sl1_ct

,Report Month,Sales Level 1,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,202206,AMERICAS,COVERAGE AT RENEWAL,4,277,1.444043
1,202206,AMERICAS,COVERAGE PROGRAM,20,772,2.590674
2,202206,AMERICAS,FIELD,688,808,85.148515
3,202206,AMERICAS,TARGETING,289,572,50.524476
4,202206,APJC,FIELD,29,59,49.152542
5,202206,APJC,TARGETING,93,313,29.712460
6,202206,EMEAR-REGION,COVERAGE PROGRAM,1,131,0.763359
7,202206,EMEAR-REGION,FIELD,59,146,40.410959
8,202206,EMEAR-REGION,TARGETING,96,202,47.524752


In [178]:
with pd.ExcelWriter(r'C:\Users\phsheari\Documents\FIN BI Data\Conversion Rate\ConversionRates_' + file_run_date + '_.xlsx') as writer:
    combined_sl1_ct.to_excel(writer, sheet_name = 'Theater_CampaignType', index=False)
    combined_sl1_sl2_ct.to_excel(writer, sheet_name = 'Theater_SL2_CampaignType', index=False)
    combined_sl1_sl2.to_excel(writer, sheet_name = 'Geo & Theater', index=False)
    combined_sl1.to_excel(writer, sheet_name = 'Theater_Only', index=False)
    combined_ct.to_excel(writer, sheet_name = 'CampaignType_Only', index=False)      

### SL1 = Sales Level 1 and SL2 = Sales Level 2 and CT = Campaign Type

In [179]:
combined_sl1_sl2_ct

,Report Month,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,202206,AMERICAS,AMERICAS_SP,FIELD,55,61,90.163934
1,202206,AMERICAS,AMERICAS_SP,TARGETING,19,22,86.363636
2,202206,AMERICAS,CANADA,COVERAGE AT RENEWAL,1,26,3.846154
3,202206,AMERICAS,CANADA,COVERAGE PROGRAM,6,24,25.000000
4,202206,AMERICAS,CANADA,FIELD,40,39,102.564103
5,202206,AMERICAS,CANADA,TARGETING,30,54,55.555556
6,202206,AMERICAS,GES,COVERAGE PROGRAM,5,85,5.882353
7,202206,AMERICAS,GES,FIELD,250,334,74.850299
8,202206,AMERICAS,GES,TARGETING,89,191,46.596859
9,202206,AMERICAS,LATIN AMERICA,COVERAGE PROGRAM,3,22,13.636364


In [71]:
# combined_sl1_sl2_ct.to_excel(r'C:\Users\phsheari\Documents\Francisco\USCOMMERCIAL_ConversionRates.xlsx', index=False)

In [72]:
overall_conversion_ratio = (overall_numerator / overall_denominator) * 100
print(overall_numerator, overall_denominator)

1081 3250


In [73]:
overall_conversion_ratio

33.26153846153846